In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


# （2022年1月27日～）組み合わせモデルを含めたMAPE表を作成する

コア数と問題サイズを両方振ったうえで出力する

# （2022年1月27日～）✅型の相違によって警告が出力されているので修正する
# （2022年1月27日～）DataFrame.append()がなくなる予定になったために表示される警告が出力されているので修正する

In [2]:
def return_MAPE_Table_DF_from_rawDF(
    rawDF, exp_var_list=[], res_var_list=[], model_name_list=[]
):
    if exp_var_list == []:
        raise ExceptionInResearchLib("説明変数として利用するカラム名のリストが空です")
    if res_var_list == []:
        raise ExceptionInResearchLib("目的変数として利用するカラム名のリストが空です")
    if model_name_list == []:
        raise ExceptionInResearchLib("構築するモデル名のリストが空です")

    function_names = list(set(rawDF["functionName"].tolist()))
    if len(function_names) == 0:
        raise ExceptionInResearchLib("与えられた生データのDFに関数名がありません")

    # 結果を格納するためのDFを作成
    # resultDF = pd.DataFrame(columns=["functionName"] + model_name_list)
    result_series_list = []

    for function_name in function_names:
        # 関数ごとの生データ
        rawDF_per_function = rawDF[rawDF["functionName"] == function_name]
        # モデルの構築
        models = Models(
            inputDF=rawDF_per_function,
            expVarColNames=exp_var_list,
            resVarColNames=res_var_list,
            targetDF=None,
            modelNames=model_name_list,
        )
        models.setUpDataBeforeCalcLr()
        models.calcLr()
        models.calcMAPE()

        # MAPEの算出
        dictCalcedMAPE = models.returnCalculatedMAPE()

        # 算出されたMAPEの数値を小数第一位までにする
        for key in dictCalcedMAPE.keys():
            # dictCalcedMAPE[key] = int(dictCalcedMAPE[key]*10)/10
            dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

        # 関数ごとの結果に格納
        dict_for_series = copy.deepcopy(dictCalcedMAPE)
        dict_for_series["functionName"] = function_name

        series = pd.Series(dict_for_series)
        # TODO:下記のappend()をconcat()にする
        # resultDF = resultDF.append(series, ignore_index=True)
        result_series_list.append(series)
    resultDF = pd.DataFrame(result_series_list)
    return resultDF


def test_return_MAPE_Table_DF_from_rawDF():
    # 構築するモデルのリスト
    model_name_list = ["modelLin", "modelIp", "modelLog"]
    # モデル構築用のDFを作成
    plotX = np.linspace(10, 20, 10)
    plotY = np.linspace(20, 30, 10)
    plotZ = np.linspace(30, 40, 10)
    function_name_list = ["function_name"] * 10
    benchmark_name_list = ["benchmark_name"] * 10
    exp_var_list = ["plotX", "plotY", "plotZ"]
    res_var_list = ["plotT"]
    plotT = 100 * plotX - 500 * plotY + 0.1 * plotZ + 40
    rawDF = pd.DataFrame(
        data={
            "benchmarkName": benchmark_name_list,
            "functionName": function_name_list,
            "plotT": plotT,
            "plotX": plotX,
            "plotY": plotY,
            "plotZ": plotZ,
        }
    )
    # テスト対象の関数からDFを取得
    returnedDF = return_MAPE_Table_DF_from_rawDF(
        rawDF=rawDF,
        exp_var_list=exp_var_list,
        res_var_list=res_var_list,
        model_name_list=model_name_list,
    )
    # modelLinが最小であることを確認
    assert returnedDF.at[0, "modelLin"] < returnedDF.at[0, "modelIp"]
    assert returnedDF.at[0, "modelLin"] < returnedDF.at[0, "modelLog"]

    # TODO:組み合わせモデル(線形+反比例)
    # 構築するモデルのリスト
    model_name_list = [
        "modelLin",
        "modelIp",
        "modelLog",
        "modelLinAndLog",
        "modelLinAndIp",
    ]
    # モデル構築用のDFを作成
    plotX = np.linspace(10, 20, 10)
    plotY = np.linspace(20, 30, 10)
    plotZ = np.linspace(30, 40, 10)
    function_name_list = ["function_name"] * 10
    benchmark_name_list = ["benchmark_name"] * 10
    exp_var_list = ["process", "plotY", "plotZ"]
    res_var_list = ["plotT"]
    plotT = 100 * plotX + -100 * plotY + 40 / plotZ + 30
    rawDF = pd.DataFrame(
        data={
            "benchmarkName": benchmark_name_list,
            "functionName": function_name_list,
            "plotT": plotT,
            "process": plotX,
            "plotY": plotY,
            "plotZ": plotZ,
        }
    )
    # テスト対象の関数からDFを取得
    returnedDF = return_MAPE_Table_DF_from_rawDF(
        rawDF=rawDF,
        exp_var_list=exp_var_list,
        res_var_list=res_var_list,
        model_name_list=model_name_list,
    )
    # modelLinAndIpが最小であることを確認
    print(returnedDF)
    for model_name in model_name_list:
        if model_name == "modelLinAndIp":
            pass
        else:
            assert returnedDF.at[0, "modelLinAndIp"] < returnedDF.at[0, model_name]

    return

In [3]:
test_return_MAPE_Table_DF_from_rawDF()

   modelLin       modelIp      modelLog  modelLinAndIp  modelLinAndLog  \
0  0.000794  1.173452e-14  6.706793e-07   1.056090e-14    4.808977e-07   

    functionName  
0  function_name  


In [4]:
# 引数として渡されたDFに最低MAPEのモデル名の列を追加する関数
# 引数 inputDF：DF[関数名(列名ではなくインデックス), モデル名1, ... , モデル名n], model_name_list
# 引数 model_name_list：モデル名が要素のリスト
# 引数 version：バージョン。1がオリジナル。
# 返値：DF[関数名(列名ではなくインデックス), モデル名1, ... , モデル名n, 最低MAPEのモデル名]


def addLowestMAPEsModelNameColumn(inputDF, model_name_list=[], version=1):

    """addLowestMAPEsModelNameColumn()の説明

    引数として渡されたDFに最低MAPEのモデル名の列を追加する関数

    Args:
        inputDF (pandas.DataFrame): DF[関数名(列名ではなくインデックス), モデル名1, ... , モデル名n]
        model_name_list (list): モデル名が要素のリスト
        version(int): バージョン。1がオリジナル。

    Returns:
        pandas.DataFrame: DF[関数名(列名ではなくインデックス), モデル名1, ... , モデル名n, 最低MAPEのモデル名]

    Raises:
        ExceptionInResearchLib: 引数に矛盾が生じている場合に発生

    Yields:
        戻り値の型: 戻り値についての説明

    Note:
        version=1の時のドキュメント消失。使うな。

    """

    if len(model_name_list) == 0 or version == 1:
        logger.warning(
            "addLowestMAPEsModelNameColumn()：version=1はすでに古い実装となっています。version=2の利用を検討してください。"
        )
        funcNames = inputDF.index.to_list()
        modelNames = inputDF.columns.to_list()

        inputDF["最適モデル"] = "lin"

        for funcName in funcNames:
            lowestInFunc_modelName = ""
            lowestInFunc = math.inf
            seriesInFunc = inputDF.loc[funcName]
            for modelName in modelNames:
                elem = seriesInFunc[modelName]
                if type(float(elem)) != type(float(lowestInFunc)):
                    logger.debug(
                        f"type(elem)={type(elem)}, type(lowestInFunc)={type(lowestInFunc)}"
                    )
                    logger.debug(f"elem={elem}, lowestInFunc={lowestInFunc}")
                if elem < lowestInFunc:
                    lowestInFunc = elem
                    lowestInFunc_modelName = modelName
                inputDF.at[funcName, "最適モデル"] = lowestInFunc_modelName

        return inputDF
    elif model_name_list != 0 and version == 2:
        func_names = inputDF.index.to_list()

        inputDF["最適モデル"] = model_name_list[0]

        for func_name in func_names:
            lowestInFunc_modelName = ""
            lowestInFunc = math.inf
            seriesInFunc = inputDF.loc[func_name]
            for model_name in model_name_list:
                elem = seriesInFunc[model_name]
                if type(float(elem)) != type(float(lowestInFunc)):
                    logger.debug(
                        f"type(elem)={type(elem)}, type(lowestInFunc)={type(lowestInFunc)}"
                    )
                    logger.debug(f"elem={elem}, lowestInFunc={lowestInFunc}")
                if elem < lowestInFunc:
                    lowestInFunc = elem
                    lowestInFunc_modelName = model_name
                inputDF.at[func_name, "最適モデル"] = lowestInFunc_modelName
        return inputDF
    else:
        ExceptionInResearchLib("addLowestMAPEsModelNameColumn()に渡された引数に矛盾が生じています")


def test_addLowestMAPEsModelNameColumn():
    # 入力を作成
    base_dict = {
        "lin": [1, 2, 2, 1],
        "ip": [2, 1, 3, 1],
        "log": [3, 3, 1, 1],
    }
    input_DF = pd.DataFrame.from_dict(data=base_dict)
    # テスト結果を手動で作成
    result_DF_sample = input_DF.copy(deep=True)
    result_DF_sample["最適モデル"] = ["lin", "ip", "log", "lin"]
    # テスト対象の関数を実行
    result_DF = addLowestMAPEsModelNameColumn(input_DF)
    # 結果を比較
    assert (result_DF_sample == result_DF).all().all()

    # 入力を作成
    base_dict = {
        "lin": [1, 2, 2, 1],
        "ip": [2, 1, 3, 1],
        "log": [3, 3, 1, 1],
    }
    input_DF = pd.DataFrame.from_dict(data=base_dict)
    # テスト結果を手動で作成
    result_DF_sample = input_DF.copy(deep=True)
    result_DF_sample["最適モデル"] = ["lin", "ip", "log", "lin"]
    # テスト対象の関数を実行
    result_DF = addLowestMAPEsModelNameColumn(
        input_DF, model_name_list=["lin", "ip", "log", "lin"], version=2
    )
    # 結果の確認
    assert input_DF["最適モデル"].to_list() == result_DF["最適モデル"].to_list()

In [5]:
test_addLowestMAPEsModelNameColumn()

In [6]:
# 引数に該当するデータからMAPE（学習データに対する一致度）を各モデルごとにまとめたデータフレームを返す関数
# benchmarkName:ベンチマーク名
# classes:ベンチマーククラスのリスト
# targetClass:不要だがベンチマーククラスを指定できる
# processes:コア数のリスト
# targetProcess:不要だがコア数を指定できる
# expVar:学習に使用する列名もリスト
# csvDirPath:CSVの保存されているディレクトリへのパス
# 返り値：列名が["関数名", "モデル1", ... , "モデルN"]となっているDF


def returnDictAboutMAPETable(
    benchmarkName,
    classes,
    targetClass,
    processes,
    targetProcess,
    expVar,
    csvDirPath,
    modelNames=["modelLin", "modelIp", "modelLog"],
):

    """returnDictAboutMAPETable()の説明

    引数に該当するデータからMAPE（学習データに対する一致度）を各モデルごとにまとめたデータフレームを返す関数

    Args:
        benchmarkName (string): ベンチマーク名
        classes (list): ベンチマーククラスのリスト
        targetClass (list): 不要だがベンチマーククラスを指定できる
        processes (list): コア数のリスト
        targetProcess (int): 不要だがコア数を指定できる
        expVar (list): 学習に使用する列名もリスト
        csvDirPath (string): CSVの保存されているディレクトリへのパス
        modelNames (list): モデル名のリスト

    Returns:
        pandas.DataFrame: 列名が["関数名", "モデル1", ... , "モデルN"]となっているDF

    """

    # データを取得
    rawDF = returnCollectedExistingData(
        benchmarkNames=[benchmarkName],
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
    )
    rawDF = addInitDataToRawDF(rawDF)
    # 結果を格納するためのDFを作成
    resultDF = pd.DataFrame(columns=["functionName"] + modelNames)
    # モデルを用いた処理を実施
    functionNames = sorted(list(set(rawDF["functionName"])))
    for functionName in functionNames:
        ## 関数ごとに生データを取得
        DFExtractedByFunction = rawDF[rawDF["functionName"] == functionName].copy()
        ## 説明変数用に問題サイズ列を数値化した列を追加する
        strListProblemSize = DFExtractedByFunction["benchmarkClass"].tolist()
        intListProblemSize = convertBenchmarkClasses_problemSizeInNPB(
            inputList=strListProblemSize
        )
        DFExtractedByFunction["intBenchmarkClass"] = intListProblemSize
        ## 3モデルでMAPEを出力
        ### 目的変数のリストを作成
        resVar = ["functionCallNum"]
        ### 回帰を行う
        #### 3モデルを同時に作成
        models = Models(
            inputDF=DFExtractedByFunction,
            expVarColNames=expVar,
            resVarColNames=resVar,
            targetDF=None,
            modelNames=modelNames,
        )
        models.setUpDataBeforeCalcLr()
        models.calcLr()
        models.calcMAPE()
        dictCalcedMAPE = models.returnCalculatedMAPE()
        #### 計算されたMAPEの数値を小数第一位までにする
        for key in dictCalcedMAPE.keys():
            dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
        #### 関数ごとの結果に格納
        dict_for_series = {
            "functionName": functionName,
        }
        for key in dictCalcedMAPE.keys():
            dict_for_series[key] = dictCalcedMAPE[key]
        series = pd.Series(dict_for_series)
        resultDF = resultDF.append(series, ignore_index=True)
    return resultDF


def test_returnDictAboutMAPETable(csvDirPath="../csv_files/"):
    ####
    # 予測を行う。一つの関数・変数（コア数・各種ベンチマーク由来の初期化変数）
    benchmarkNames = ["cg"]
    benchmarkName = "cg"
    classes = ["S", "W", "A", "B", "C", "D", "E", "F"]
    targetClass = "F"
    processes = [128]
    targetProcess = 256

    # データを取得
    rawDF = returnCollectedExistingData(
        benchmarkNames=benchmarkNames,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
    )
    rawDF = addInitDataToRawDF(rawDF)

    # 結果(初期変数含む)を格納するためのDF
    # resultDictMulti = pd.DataFrame(
    #     columns=["functionName", "modelLin", "modelIp", "modelLog"]
    # )
    result_series_list_multi = []
    # 結果(初期変数含まない)を格納するためのDF
    # resultDictSingle = pd.DataFrame(
    #     columns=["functionName", "modelLin", "modelIp", "modelLog"]
    # )
    result_series_list_single = []

    # モデルを用いた処理を実施
    functionNames = sorted(list(set(rawDF["functionName"])))
    for functionName in functionNames:
        ##関数ごとにrawDFを抽出
        DFExtractedByFunction = rawDF[rawDF["functionName"] == functionName].copy()
        ## 説明変数用に問題サイズ列を数値化した列を追加する
        strListProblemSize = DFExtractedByFunction["benchmarkClass"].tolist()
        intListProblemSize = convertBenchmarkClasses_problemSizeInNPB(
            inputList=strListProblemSize
        )
        DFExtractedByFunction["intBenchmarkClass"] = intListProblemSize
        ## 3モデルでMAPEを出力
        ### 目的変数のリストを作成
        resVar = ["functionCallNum"]
        ### 重回帰分析（初期変数を含めた）
        #### 列名のリストをDFから取得
        expVarMulti = DFExtractedByFunction.columns.tolist()
        #### 除外する列名を除く
        for removeElement in [
            "functionName",
            "functionCallNum",
            "benchmarkName",
            "benchmarkClass",
        ]:
            expVarMulti.remove(removeElement)
        #### 3モデル（線形・反比例・対数）を同時に作成
        threeModelsByMulti = Models(
            inputDF=DFExtractedByFunction,
            expVarColNames=expVarMulti,
            resVarColNames=resVar,
            targetDF=None,
            modelNames=["modelLin", "modelIp", "modelLog"],
        )
        threeModelsByMulti.setUpDataBeforeCalcLr()
        threeModelsByMulti.calcLr()
        threeModelsByMulti.calcMAPE()
        dictCalcedMAPE = threeModelsByMulti.returnCalculatedMAPE()
        # dictCalcedMAPEの数値を小数第一位までにする
        for key in dictCalcedMAPE.keys():
            dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
        #### 関数ごとの結果に格納
        seriesMulti = pd.Series(
            {
                "functionName": functionName,
                "modelLin": dictCalcedMAPE["modelLin"],
                "modelIp": dictCalcedMAPE["modelIp"],
                "modelLog": dictCalcedMAPE["modelLog"],
            }
        )
        # resultDictMulti = resultDictMulti.append(seriesMulti, ignore_index=True)
        result_series_list_multi.append(seriesMulti)

        ### 単回帰分析（初期変数を含めない）
        expVarSingle = ["process", "intBenchmarkClass"]
        threeModelsBySingle = Models(
            inputDF=DFExtractedByFunction,
            expVarColNames=expVarSingle,
            resVarColNames=resVar,
            targetDF=None,
            modelNames=["modelLin", "modelIp", "modelLog"],
        )
        threeModelsBySingle.setUpDataBeforeCalcLr()
        threeModelsBySingle.calcLr()
        threeModelsBySingle.calcMAPE()
        dictCalcedMAPE = threeModelsBySingle.returnCalculatedMAPE()
        # dictCalcedMAPEの数値を小数第一位までにする
        for key in dictCalcedMAPE.keys():
            dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
        #### 関数ごとの結果に格納
        seriesSingle = pd.Series(
            {
                "functionName": functionName,
                "modelLin": dictCalcedMAPE["modelLin"],
                "modelIp": dictCalcedMAPE["modelIp"],
                "modelLog": dictCalcedMAPE["modelLog"],
            }
        )
        # resultDictSingle = resultDictSingle.append(seriesSingle, ignore_index=True)
        result_series_list_single.append(seriesSingle)

    resultDictMulti = pd.DataFrame(result_series_list_multi)
    resultDictSingle = pd.DataFrame(result_series_list_single)

    resultDictMultiFromFunc = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=targetClass,
        processes=processes,
        targetProcess=targetProcess,
        expVar=expVarMulti,
        csvDirPath=csvDirPath,
    )
    pd.testing.assert_frame_equal(
        resultDictMulti, resultDictMultiFromFunc, check_dtype=False
    )
    resultDictSingleFromFunc = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=targetClass,
        processes=processes,
        targetProcess=targetProcess,
        expVar=expVarSingle,
        csvDirPath=csvDirPath,
    )
    pd.testing.assert_frame_equal(
        resultDictSingle, resultDictSingleFromFunc, check_dtype=False
    )

    return 0

In [7]:
test_returnDictAboutMAPETable(csvDirPath="./csv_files/")

/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated a

0

# （2022年1月19日～2022年1月26日）最適モデルをまとめた表を作成する処理の確認

`returnDictAboutMAPETable()` の問題に伴い、最適モデルの組み合わせをまとめた表の結果が怪しい。修正する。

In [8]:
benchmark_name = "cg"

fix_class = ["B"]
fix_process = [128]

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

dict_column_fix_process = {"最適モデル": "最適モデル（コア数固定）"}
dict_column_fix_core = {"最適モデル": "最適モデル（問題サイズ固定）"}

date = "2022年1月22日"

# 問題サイズを固定してコア数を振る
fix_class_rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=fix_class,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var = fix_class_rawDF.columns.tolist()
exp_var.remove("functionName")
exp_var.remove("functionCallNum")
exp_var.remove("benchmarkName")
exp_var.remove("benchmarkClass")
res_var = ["functionCallNum"]

fix_class_MAPE_table = return_MAPE_Table_DF_from_rawDF(
    rawDF=fix_class_rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)

# コア数を固定して問題サイズを振る
fix_process_rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=fix_process,
    csv_dir_path="./csv_files/",
)

fix_process_MAPE_table = return_MAPE_Table_DF_from_rawDF(
    rawDF=fix_process_rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)

# 2条件(問題サイズ固定・コア数固定)で取得したMAPE表に最低MAPE値を記録した列を追加する
fix_process_MAPE_table = addLowestMAPEsModelNameColumn(
    fix_process_MAPE_table, model_name_list=model_names_list, version=2
)
fix_process_MAPE_table = fix_process_MAPE_table.set_index("functionName")
fix_class_MAPE_table = addLowestMAPEsModelNameColumn(
    fix_class_MAPE_table, model_name_list=model_names_list, version=2
)
fix_class_MAPE_table = fix_class_MAPE_table.set_index("functionName")

fix_process_MAPE_table_bestModel = (
    fix_process_MAPE_table.loc[:, ["最適モデル"]]
    .rename(columns={"最適モデル": "最適モデル(コア数固定)"})
    .replace(dict_column_names_JP)
)
fix_core_MAPE_table_bestModel = (
    fix_class_MAPE_table.loc[:, ["最適モデル"]]
    .rename(columns={"最適モデル": "最適モデル(問題サイズ固定)"})
    .replace(dict_column_names_JP)
)

print(
    pd.merge(
        fix_core_MAPE_table_bestModel,
        fix_process_MAPE_table_bestModel,
        how="outer",
        right_index=True,
        left_index=True,
    )
    .sort_index()
    .style.to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}における最適モデル",
        # index=True,
        label=f"{date}bestModelCombinationIn{benchmark_name.upper()}",
    )
)

\begin{table}
\caption{ベンチマークプログラムCGにおける最適モデル}
\label{2022年1月22日bestModelCombinationInCG}
\begin{tabular}{lll}
 & 最適モデル(問題サイズ固定) & 最適モデル(コア数固定) \\
functionName &  &  \\
.TAU_application & 線形モデル & 線形モデル \\
ALLOC_SPACE & 線形モデル & 線形モデル \\
CG & 線形モデル & 線形モデル \\
CONJ_GRAD & 反比例モデル & 対数モデル \\
ICNVRT & 反比例モデル & 線形モデル \\
INITIALIZE_MPI & 線形モデル & 線形モデル \\
MAKEA & 線形モデル & 線形モデル \\
MPI_Barrier() & 線形モデル & 線形モデル \\
MPI_Bcast() & 線形モデル & 線形モデル \\
MPI_Comm_rank() & 線形モデル & 線形モデル \\
MPI_Comm_size() & 線形モデル & 線形モデル \\
MPI_Finalize() & 線形モデル & 線形モデル \\
MPI_Init() & 線形モデル & 線形モデル \\
MPI_Irecv() & 対数モデル & 対数モデル \\
MPI_Reduce() & 線形モデル & 線形モデル \\
MPI_Send() & 対数モデル & 対数モデル \\
MPI_Wait() & 対数モデル & 対数モデル \\
SETUP_PROC_INFO & 線形モデル & 線形モデル \\
SETUP_SUBMATRIX_INFO & 線形モデル & 線形モデル \\
SPARSE & 線形モデル & 線形モデル \\
SPRNVC & 反比例モデル & 線形モデル \\
VECSET & 反比例モデル & 線形モデル \\
\end{tabular}
\end{table}



# （2022年1月19日～）MAPEをまとめた表を作成する関数の修正もしくは別バージョンの作成

`returnDictAboutMAPETable()`は生データを取得する機能とMAPE表を作成する機能がまとまっている。

* ✅生データを取得する機能を持つ関数の作成
    * 引数
        * ベンチマーク名
        * 問題サイズのリスト
        * コア数のリスト
        * 問題サイズごとの初期パラメータの付与の有無
* ✅生データからMAPEを作成する関数の作成
    * 引数
        * 既存の`returnDictAboutMAPETable()`における引数で生データの取得のみに必要な引数は除く

# （2022年1月15日～2022年1月18日）組み合わせモデルの検証

問題サイズ固定時とコア数固定時での最適モデルから予測される組み合わせモデルが、実際に選択される組み合わせモデルと異なる。したがって、その異なっている原因がプログラムの実装ミスに由来するものなのかどうかを検証する。

やること（TODO）

1. ✅問題サイズ固定時とコア数固定時とでは異なるモデルを算出する関数をDFとして抽出
2. ✅各関数でモデルを構築
3. ✅出力した表の列名を英語から日本語にする
4. ✅現行で最適となっているモデルと最適となるはずの組み合わせモデルの違いを探す
    1. returnDictAboutMAPETable()に「ベンチマークプログラムごとに異なる問題サイズ由来の初期変数を追加する処理が含まれていない」問題があったのでそれをやらないといけない

In [9]:
resultsDict = {}

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128]
benchmark_name = "cg"

# ベンチマークプログラム名のリスト
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIp": "反比例",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLog": "対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月15日"

target_function_names = ["CONJ_GRAD", "ICNVRT", "SPRNVC", "VECSET"]

expVar = returnExplanatoryVariablesList(
    benchmarkName=benchmark_name,
    classes=classes,
    processes=processes,
    csvDirPath=csvDirPath,
    baseExpVar=True,
    initExpVar=True,
)

rawDF = returnCollectedExistingData(
    benchmarkNames=[benchmark_name],
    classes=classes,
    processes=processes,
    csvDirPath=csvDirPath,
)

target_functions_rawDF = rawDF[
    (rawDF["functionName"] == target_function_names[0])
    | (rawDF["functionName"] == target_function_names[1])
    | (rawDF["functionName"] == target_function_names[2])
    | (rawDF["functionName"] == target_function_names[3])
]

# 説明変数用に問題サイズを数値化した列を追加
strListProblemSize = target_functions_rawDF["benchmarkClass"].tolist()
intListProblemSize = convertBenchmarkClasses_problemSizeInNPB(
    inputList=strListProblemSize
)
target_functions_rawDF["intBenchmarkClass"] = intListProblemSize
# 説明変数用に問題サイズ由来のほかの数値を保持する列を追加
target_functions_rawDF = addInitDataToRawDF(target_functions_rawDF)

# 目的変数の列名を保持したリスト
res_var = ["functionCallNum"]
# 説明変数の列名を保持したリスト
exp_var = target_functions_rawDF.columns.tolist()
exp_var.remove("functionName")
exp_var.remove("benchmarkName")
exp_var.remove("benchmarkClass")
print(exp_var)

# 結果を格納するためのDFを作成
# resultDF = pd.DataFrame(columns=["functionName"] + model_names_list)
# print(resultDF)
result_series_list = []

for target_function_name in target_function_names:
    # 関数ごとに生データを取得
    target_function_rawDF = target_functions_rawDF[
        target_functions_rawDF["functionName"] == target_function_name
    ]

    # モデルの構築
    models = Models(
        inputDF=target_function_rawDF,
        expVarColNames=exp_var,
        resVarColNames=res_var,
        targetDF=None,
        modelNames=model_names_list,
    )
    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    # MAPEの算出
    dictCalcedMAPE = models.returnCalculatedMAPE()

    # 算出されたMAPEの数値を小数第一位までにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10

    # 関数ごとの結果に格納
    dict_for_series = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = target_function_name

    series = pd.Series(dict_for_series)
    result_series_list.append(series)
    # resultDF = resultDF.append(series, ignore_index=True)
resultDF = pd.DataFrame(result_series_list)
resultDF = resultDF.rename(columns=dictColumnNamesJP)

print(
    resultDF.style.to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}における特定の関数でのMAPE",
        # index=False,
        label=f"{date}bestModelCombinationIn{benchmark_name.upper()}inTargetFunctions",
    )
)

# df = returnDictAboutMAPETable(
#     benchmarkName=benchmark_name,
#     classes=classes,
#     targetClass=None,
#     processes=processes,
#     targetProcess=None,
#     expVar=expVar,
#     csvDirPath=csvDirPath,
#         modelNames=model_names_list,
# )
# df

/tmp/ipykernel_1167/2464667175.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_functions_rawDF["intBenchmarkClass"] = intListProblemSize
/tmp/ipykernel_1167/2694063525.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawDF[dictKey] = columnDict[dictKey]
/tmp/ipykernel_1167/2694063525.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

['functionCallNum', 'process', 'intBenchmarkClass', 'na', 'nonzer', 'niter', 'shift']
\begin{table}
\caption{ベンチマークプログラムCGにおける特定の関数でのMAPE}
\label{2022年1月15日bestModelCombinationInCGinTargetFunctions}
\begin{tabular}{lrrrrrrrrrl}
 & 線形 & 反比例 & 対数 & 線形＆反比例 & 線形＆対数 & 反比例＆線形 & 反比例＆対数 & 対数＆線形 & 対数＆反比例 & 関数名 \\
0 & 0.000000 & 5.300000 & 3.100000 & 5.000000 & 2.600000 & 0.000000 & 2.700000 & 0.000000 & 5.900000 & CONJ_GRAD \\
1 & 0.000000 & 308.300000 & 452.700000 & 435.000000 & 594.200000 & 0.000000 & 218.300000 & 0.000000 & 537.000000 & ICNVRT \\
2 & 0.000000 & 215.600000 & 170.200000 & 275.900000 & 225.300000 & 0.000000 & 95.600000 & 0.000000 & 310.900000 & SPRNVC \\
3 & 0.000000 & 215.600000 & 170.200000 & 275.900000 & 225.300000 & 0.000000 & 95.600000 & 0.000000 & 310.900000 & VECSET \\
\end{tabular}
\end{table}



# （～2022年1月14日）組み合わせモデルにおいて最適なモデルを出力する

| 関数名    | コア数固定での最適モデル | 問題サイズ固定での最適モデル | 
| --------- | ------------------------ | ---------------------------- | 
| func_name | model_name               | model_name                   | 

上記のような表を作成する。

In [10]:
resultsList = []

# コア数を128に固定し、問題サイズを振る
classes_fix_core = ["A", "B", "C", "D", "E", "F"]
processes_fix_core = [128]
# 問題サイズをBに固定し、コア数を振る
classes_fix_class = ["B"]
processes_fix_class = [2, 4, 8, 16, 32, 64, 128]
# ベンチマークプログラム名のリスト
benchmark_names = ["cg"]
# benchmark_names = ["cg", "ep", "ft", "is", "lu", "mg"]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

dict_column_fix_core = {"最適モデル": "最適モデル（コア数固定）"}
dict_column_fix_class = {"最適モデル": "最適モデル（問題サイズ固定）"}

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月10日"

# 方針

1. ✅「最適モデル」列を追加する関数を作成
2. ✅作成した関数をループ内で使用
3. ✅ループ内で、「コア数を128に固定し、問題サイズを振る」と「問題サイズをBに固定し、コア数を振る」
4. 2つの実験結果を結合させる
    1. ✅それぞれの実験結果から列「最適モデル」を取得
    2. ✅「関数名| 最適モデル(問題サイズ固定)| 最適モデル(コア数固定)」となるように表を操作
5. ✅「関数名｜モデル０｜．．．｜モデルN｜最適モデル」となる表をベンチマークCGで作成
6. ✅「5.」でやったことを他のベンチマークプログラムで実施

In [11]:
# （～2022年1月14日）コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDict_fix_core = {}
resultDict_fix_class = {}

# resultDictAllWithoutInit = {}

for benchmarkName in benchmark_names:
    print(f"% _____benchmarkName={benchmarkName}_____\n\n\n")

    # 問題サイズを固定し、コア数を振る
    expVar_fix_class = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes_fix_class,
        processes=processes_fix_class,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )

    df_fix_class = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes_fix_class,
        targetClass=None,
        processes=processes_fix_class,
        targetProcess=None,
        expVar=expVar_fix_class,
        csvDirPath=csvDirPath,
    )
    df_fix_class = df_fix_class.rename(columns=dictColumnNamesJP)
    df_fix_class = df_fix_class.set_index("関数名")
    result_fix_class = addLowestMAPEsModelNameColumn(df_fix_class).rename(
        columns=dict_column_fix_class
    )
    resultDict_fix_class[benchmarkName.upper()] = result_fix_class

    # コア数を固定し、問題サイズを振る
    expVar_fix_core = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes_fix_core,
        processes=processes_fix_core,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )

    df_fix_core = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes_fix_core,
        targetClass=None,
        processes=processes_fix_core,
        targetProcess=None,
        expVar=expVar_fix_core,
        csvDirPath=csvDirPath,
    )
    df_fix_core = df_fix_core.rename(columns=dictColumnNamesJP)
    df_fix_core = df_fix_core.set_index("関数名")
    result_fix_core = addLowestMAPEsModelNameColumn(df_fix_core).rename(
        columns=dict_column_fix_core
    )
    resultDict_fix_core[benchmarkName.upper()] = result_fix_core

    # 作成した表を結合する
    merged_DF = pd.merge(df_fix_class, df_fix_core)
    dropped_column_name = merged_DF.columns.to_list()
    print(
        pd.merge(
            result_fix_class.drop(model_names_list_jp, axis=1),
            result_fix_core.drop(model_names_list_jp, axis=1),
            how="outer",
            right_index=True,
            left_index=True,
        ).style.to_latex(
            caption=f"ベンチマークプログラム{benchmarkName.upper()}における最適モデル",
            label=f"{date}bestModelCombinationIn{benchmarkName.upper()}",
        )
    )

% _____benchmarkName=cg_____





/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated a

\begin{table}
\caption{ベンチマークプログラムCGにおける最適モデル}
\label{2022年1月10日bestModelCombinationInCG}
\begin{tabular}{lll}
 & 最適モデル（問題サイズ固定） & 最適モデル（コア数固定） \\
関数名 &  &  \\
.TAU_application & 線形モデル & 線形モデル \\
ALLOC_SPACE & 線形モデル & 線形モデル \\
CG & 線形モデル & 線形モデル \\
CONJ_GRAD & 反比例モデル & 線形モデル \\
ICNVRT & 反比例モデル & 線形モデル \\
INITIALIZE_MPI & 線形モデル & 線形モデル \\
MAKEA & 線形モデル & 線形モデル \\
MPI_Barrier() & 線形モデル & 線形モデル \\
MPI_Bcast() & 線形モデル & 線形モデル \\
MPI_Comm_rank() & 線形モデル & 線形モデル \\
MPI_Comm_size() & 線形モデル & 線形モデル \\
MPI_Finalize() & 線形モデル & 線形モデル \\
MPI_Init() & 線形モデル & 線形モデル \\
MPI_Irecv() & 対数モデル & 線形モデル \\
MPI_Reduce() & 線形モデル & 線形モデル \\
MPI_Send() & 対数モデル & 線形モデル \\
MPI_Wait() & 対数モデル & 線形モデル \\
SETUP_PROC_INFO & 線形モデル & 線形モデル \\
SETUP_SUBMATRIX_INFO & 線形モデル & 線形モデル \\
SPARSE & 線形モデル & 線形モデル \\
SPRNVC & 反比例モデル & 線形モデル \\
VECSET & 反比例モデル & 線形モデル \\
\end{tabular}
\end{table}



In [12]:
resultsDict = {}

# コア数を128に固定し、問題サイズを振る
classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128]
# ベンチマークプログラム名のリスト
# benchmark_names = ["cg", "ep", "ft", "is", "lu", "mg"]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月10日"

In [13]:
# コア数と問題サイズの両方を振る

for benchmarkName in benchmark_names:
    print(f"% _____benchmarkName={benchmarkName}_____\n\n\n")

    # TODO: 1.「関数名｜モデル０｜．．．｜モデルN｜最適モデル」となる表をベンチマークCGで作成
    # TODO: 2.「1.」でやったことを他のベンチマークプログラムで実施

    expVar = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )
    print(f"expVar={expVar}")

    df = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVar,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )

    df = df.rename(columns=dictColumnNamesJP)
    df = df.set_index("関数名")
    result = addLowestMAPEsModelNameColumn(df)

    resultsDict[benchmarkName.upper()] = result

    print(
        result.style.to_latex(
            caption=f"ベンチマークプログラム{benchmarkName.upper()}における各モデルでのMAPE",
            label=f"{date}allModelsMAPEin{benchmarkName.upper()}",
        )
    )
#     print(
#         pd.merge(
#             result_fix_class.drop(model_names_list_jp, axis=1),
#             result_fix_core.drop(model_names_list_jp, axis=1),
#             how="outer",
#             right_index=True,
#             left_index=True,
#         ).style.to_latex(caption=f"ベンチマークプログラム{benchmarkName.upper()}における最適モデル", label="bestModelCombinationInCG")
#     )

% _____benchmarkName=cg_____



expVar=['process', 'intBenchmarkClass', 'na', 'nonzer', 'niter', 'shift']


/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDF = resultDF.append(series, ignore_index=True)
/tmp/ipykernel_1167/2128401744.py:89: FutureWarning: The frame.append method is deprecated a

\begin{table}
\caption{ベンチマークプログラムCGにおける各モデルでのMAPE}
\label{2022年1月10日allModelsMAPEinCG}
\begin{tabular}{lllllllllll}
 & 線形 & 反比例 & 対数 & 線形＆反比例 & 線形＆対数 & 反比例＆線形 & 反比例＆対数 & 対数＆線形 & 対数＆反比例 & 最適モデル \\
関数名 &  &  &  &  &  &  &  &  &  &  \\
.TAU_application & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 線形 \\
ALLOC_SPACE & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 線形 \\
CG & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 線形 \\
CONJ_GRAD & 14.400000 & 15.500000 & 16.000000 & 14.400000 & 14.400000 & 15.500000 & 15.500000 & 16.000000 & 16.000000 & 線形 \\
ICNVRT & 471.800000 & 727.800000 & 768.900000 & 471.800000 & 471.800000 & 727.800000 & 727.800000 & 768.900000 & 768.900000 & 線形 \\
INITIALIZE_MPI & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 線形 \\
MAKEA & 0.000000 & 0.000000 & 0.0